In [1]:
# Импорт всех необходимых библиотек
!pip install catboost xgboost

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from catboost import CatBoostClassifier
import xgboost as xgb

data = pd.read_excel('Данные_для_курсовои_Классическое_МО.xlsx').drop(columns=['Unnamed: 0'])
data.dropna(inplace=True)

# Целевая переменная — SI > 8
si_threshold = 8
y = (data['SI'] > si_threshold).astype(int)
X = data.drop(['IC50, mM', 'CC50, mM', 'SI'], axis=1)

# Проверка баланса классов
print("Распределение классов:\n", y.value_counts())

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Масштабирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Распределение классов:
 SI
0    642
1    356
Name: count, dtype: int64


In [2]:
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)
y_pred_logreg = logreg.predict(X_test_scaled)

print("Logistic Regression:")
print("Accuracy:", accuracy_score(y_test, y_pred_logreg))
print("F1:", f1_score(y_test, y_pred_logreg))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_logreg))

Logistic Regression:
Accuracy: 0.695
F1: 0.5734265734265734
ROC-AUC: 0.6685773556065073


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
cat_model = CatBoostClassifier(
    iterations=1000,
    depth=6,
    learning_rate=0.05,
    random_seed=42,
    verbose=0
)
cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_test)

print("CatBoost:")
print("Accuracy:", accuracy_score(y_test, y_pred_cat))
print("F1:", f1_score(y_test, y_pred_cat))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_cat))

CatBoost:
Accuracy: 0.67
F1: 0.5352112676056338
ROC-AUC: 0.6396986570586308


In [4]:
xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.05,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost:")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("F1:", f1_score(y_test, y_pred_xgb))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_xgb))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:48:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost:
Accuracy: 0.72
F1: 0.6056338028169014
ROC-AUC: 0.694289769625505


In [5]:
results_si_8 = pd.DataFrame({
    'Модель': ['Logistic Regression', 'CatBoost', 'XGBoost'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_logreg),
        accuracy_score(y_test, y_pred_cat),
        accuracy_score(y_test, y_pred_xgb)
    ],
    'F1': [
        f1_score(y_test, y_pred_logreg),
        f1_score(y_test, y_pred_cat),
        f1_score(y_test, y_pred_xgb)
    ],
    'ROC-AUC': [
        roc_auc_score(y_test, y_pred_logreg),
        roc_auc_score(y_test, y_pred_cat),
        roc_auc_score(y_test, y_pred_xgb)
    ]
})

display(results_si_8)

,Модель,Accuracy,F1,ROC-AUC
0,Logistic Regression,0.695,0.573427,0.668577
1,CatBoost,0.670,0.535211,0.639699
2,XGBoost,0.720,0.605634,0.694290


Лучший результат по всем метрикам показывает XGBoost:

У него самая высокая точность (72%) и самый сбалансированный F1 (0.606), что говорит о хорошем балансе между False Positives и False Negatives.

Также у него самая высокая ROC-AUC (0.694) — это означает, что модель уверенно различает два класса.

---

Логистическая регрессия выступила неожиданно достойно:

Несмотря на простоту, её Accuracy — почти такая же, как у XGBoost.

Это может говорить о том, что в признаках уже заложена неплохая линейная зависимость, и задача не слишком "сложная" для модели.

---

CatBoost отстал на этом этапе:

В других задачах он давал лучшие или сравнимые результаты, но здесь он показал самую низкую точность и F1.

Возможно, это связано с дисбалансом классов или с тем, что CatBoost "переподстраивается" под шумные признаки.

    Для классификации порога SI > 8 логистическая регрессия работает прилично, но XGBoost
    
    даёт более уверенные результаты, особенно если нужно максимально корректно классифицировать
    потенциальные лекарства.

    CatBoost, скорее всего, не подходит для этой задачи
    без дополнительной настройки или отбора признаков.